In [8]:
# Load Packages 
import pandas as pd 
import os
import glob
import pandas as pd
import sklearn
from sklearn.linear_model import LinearRegression
import numpy as np
import warnings
from pandas.core.common import SettingWithCopyWarning

warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)


In [4]:
# Indonesia Data including '[Units]' Data 
indonesia_full['[Units]'].value_counts()

NameError: name 'indonesia_full' is not defined

In [9]:
# Path for missing data directories - each indicator is saved as a seperate CSV 
path_g = '../../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Guatemala/'
path_i = '../../../Data/Data_preprocessed_for_PCA/Filtered_indicators_with_eligible_measurements_missing_data/Indonesia/'

extension = 'csv'

# generate path list for guatemala
guatemala_paths = glob.glob(path_g+'*.{}'.format(extension))

# generate csv list for guatemala 
guatemala_list = []
for string in guatemala_paths: 
    string = string.replace(path_g,'')
    guatemala_list.append(string)


# generate path list for indonesia 
indonesia_paths = glob.glob(path_i+'*.{}'.format(extension))

# generate csv list for indonesia 
indonesia_list = []
for string in indonesia_paths: 
    string = string.replace(path_i,'')
    indonesia_list.append(string)
indonesia_list[1:10]

['Indicator13.1.2.csv',
 'Indicator8.8.2.csv',
 'Indicator17.9.1.csv',
 'Indicator13.1.1.csv',
 'Indicator3.b.1.csv',
 'Indicator11.6.2.csv',
 'Indicator10.4.1.csv',
 'Indicator6.5.1.csv',
 'Indicator1.a.2.csv']

In [19]:
# Indonesia Imputation Code - Linear Regression 
# Read in full dataset with series code and units - this df includes non-disaggregated indicator measures' series codes and their units 
indonesia_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Indonesia%20Data%20Without%20Disaggregation.csv')
series_units = indonesia_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

# iteratively impute missing data - this for loop will include notes; guatemala follows the same structure (notes in this python chunk only)
path_i_out = '../../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Indonesia/'
count = 0 
count_2 = 0 
for csv in indonesia_list: 
    df = pd.read_csv(path_i+csv)
    for col in df.loc[:, df.columns != 'Year']:
        count_2 = count_2 + 1
        if (df[col].isnull().sum(axis=0) == 0):
            df[str(col)+'_new'] = df[col]
            df = df.drop(columns = [str(col)])

        elif (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) >= 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            
            # Select non missing rows to train a linear regression model 
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            
            # impute for missing rows based on trained model. 
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            # set lower bound of 0 
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            
            # logical upper bounds baased on unit type - 'per n' sets max at n, percent sets max at 100, etc.
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            
            # add this imputed data into a new column, and drop original column for this new df. 
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        
        # drop columns with 1 or fewer records 
        elif df[col].notnull().sum(axis=0) <= 1:
            print(str(col))
            count = count + 1
            print('columns with 1 or fewer records: ', count)
            print(count_2)
            print('proportion of columns with 1 or fewer records: ', count/count_2)
            
            df = df.drop(columns = [str(col)])
            
    df.to_csv(path_i_out + csv)

SG_STT_NSDSFDGVT
columns with 1 or fewer records:  1
31
proportion of columns with 1 or fewer records:  0.03225806451612903
SG_STT_NSDSFDDNR
columns with 1 or fewer records:  2
32
proportion of columns with 1 or fewer records:  0.0625
SG_STT_NSDSFDOTHR
columns with 1 or fewer records:  3
33
proportion of columns with 1 or fewer records:  0.09090909090909091
VC_DSR_HOLH
columns with 1 or fewer records:  4
58
proportion of columns with 1 or fewer records:  0.06896551724137931
VC_DSR_CDAN
columns with 1 or fewer records:  5
60
proportion of columns with 1 or fewer records:  0.08333333333333333
VC_DSR_HFDN
columns with 1 or fewer records:  6
61
proportion of columns with 1 or fewer records:  0.09836065573770492
VC_DSR_EFDN
columns with 1 or fewer records:  7
62
proportion of columns with 1 or fewer records:  0.11290322580645161
VC_DSR_CDYN
columns with 1 or fewer records:  8
63
proportion of columns with 1 or fewer records:  0.12698412698412698
VC_DSR_BSDN
columns with 1 or fewer records: 

In [21]:
# Guatemala Imputation Code - Linear Regression 

guatemala_full = pd.read_csv('https://raw.githubusercontent.com/PeishanLi/G5055_Practicum_Project2/main/Data/Guatmala%20Data%20Without%20Disaggregation.csv')
series_units = guatemala_full[['SeriesCode','[Units]']]
series_units = series_units.drop_duplicates()

path_g_out = '../../../Data/Data_preprocessed_for_PCA/Indicators_with_imputation/Guatemala/'
count = 0 
count_2 = 0 

for csv in guatemala_list: 
    df = pd.read_csv(path_g+csv)
    for col in df.loc[:, df.columns != 'Year']:
        count_2 = count_2 + 1
        if (df[col].isnull().sum(axis=0) == 0):
            df[str(col)+'_new'] = df[col]
            df = df.drop(columns = [str(col)])
        elif (df[col].notnull().sum(axis=0) > 1) & (df[col].isnull().sum(axis=0) >= 1):
            missing_rows = df[df[col].isnull()]
            nonmissing_rows = df[df[col].notnull()]
            missing_years = missing_rows[['Year']]
            y = nonmissing_rows[col]
            X = nonmissing_rows[['Year']]
            lm = LinearRegression().fit(X,y)
            missing_rows["inferred_col"] = missing_rows.apply(lambda _: '', axis=1)
            missing_rows["inferred_col"] = lm.predict(missing_years)
            missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(lower=0)
            for i in np.logspace(0,10,10):
                if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PER_'+str(i)+'_POP') is True: 
                    missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=i)
            if (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='PERCENT') is True:
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=100)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='SCORE') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=10)
            elif (series_units[series_units['SeriesCode']==str(col)]['[Units]']=='Ratio') is True: 
                missing_rows["inferred_col"] = missing_rows["inferred_col"].clip(upper=1)
            missing_rows = missing_rows[["Year","inferred_col"]]
            df = df.merge(missing_rows, on='Year', how ='left')
            df[str(col)+'_new'] = np.where(df[col].isnull(),df["inferred_col"],df[col])
            df = df.drop(columns = ["inferred_col", str(col)])
        elif df[col].notnull().sum(axis=0) <= 1:
            count = count + 1
            print('count: ', count)
            print(count_2)
            print('proportion of columns with 1 or fewer records: ', count/count_2)
            df = df.drop(columns = [str(col)])
    df.to_csv(path_g_out + csv)
    

count:  1
28
proportion of columns with 1 or fewer records:  0.03571428571428571
count:  2
29
proportion of columns with 1 or fewer records:  0.06896551724137931
count:  3
102
proportion of columns with 1 or fewer records:  0.029411764705882353
count:  4
180
proportion of columns with 1 or fewer records:  0.022222222222222223
count:  5
181
proportion of columns with 1 or fewer records:  0.027624309392265192


In [23]:
20/(181+335)

0.03875968992248062